Import packages 

In [1]:
import pandas as pd
import seaborn as sns 
from matplotlib import pyplot as plt
import numpy as np
from plotly import express as px
import sqlite3
from sklearn.linear_model import LinearRegression
from plotly.io import write_html


Part 1

Build a database and then send the temperature data to a table called "temperatures" in the database.

In [2]:
conn = sqlite3.connect("temps.db")

df = pd.read_csv("temps.csv")

# Make the dataframe into our desired format

def prepare_df(df):
    """
    The input is a dataframe and the output is a dataframe with proper column names and normalized temperature degree.
    """
    df = df.set_index(keys=["ID", "Year"])
    df = df.stack()
    df = df.reset_index()
    df = df.rename(columns = {"level_2"  : "Month" , 0 : "Temp"})
    df["Month"] = df["Month"].str[5:].astype(int)
    df["Temp"]  = df["Temp"] / 100
    return(df)


df_iter = pd.read_csv("temps.csv", chunksize = 100000)
for df in df_iter:
    # Using a for loop here to send data into database by chunksize of 100000 with modification explained below
    df = prepare_df(df)
    
    # Here, I added "FIPS" as a new column for the temperature dataframe, namely, df. The reason I did that is to merge this data frame with another one that contains country information later with SQL query. 
    df["FIPS"] = df["ID"].str[0:2]
    # I added a new column called "Date", this new column is formed by adding the year and the month columns together and change the format to the standard datetime format.
    df["Date"] = pd.to_datetime(df["Year"].astype(str) + "-" + df["Month"].astype(str))
    
    df.to_sql("temperatures", conn, if_exists = "append", index = False)

Get the countries data and send the countries data into a new table called "countries" in the database.

In [3]:


countries_url = "https://raw.githubusercontent.com/mysociety/gaze/master/data/fips-10-4-to-iso-country-codes.csv"
countries = pd.read_csv(countries_url)
#Rename the FIPS 10-4 tol FIPS for short, and also I did this to avoid potential problem when doing SQL query.

# send data into database created ealier

countries = countries.rename(columns = {"FIPS 10-4" : "FIPS","Name" : "Country"})
countries.to_sql("countries", conn, if_exists = "replace", index = False)

/opt/anaconda3/envs/PIC16B/lib/python3.7/site-packages/pandas/core/generic.py:2789: UserWarning:

The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.



Get the stations data and send the stations data into a new table called "stations" in the database.

In [4]:

url = "https://raw.githubusercontent.com/PhilChodrow/PIC16B/master/datasets/noaa-ghcn/station-metadata.csv"
stations = pd.read_csv(url)

# send data into database created ealier
stations.to_sql("stations", conn, if_exists = "replace", index = False)

To verify that the data read in from url and CSV file previously are properly sent to and stored in the database.

In [5]:
cursor = conn.cursor()
# SQL command 
cursor.execute("SELECT sql FROM sqlite_master WHERE type='table';")

for result in cursor.fetchall():
    print(result[0])
    

CREATE TABLE "temperatures" (
"ID" TEXT,
  "Year" INTEGER,
  "Month" INTEGER,
  "Temp" REAL,
  "FIPS" TEXT,
  "Date" TIMESTAMP
)
CREATE TABLE "countries" (
"FIPS" TEXT,
  "ISO 3166" TEXT,
  "Country" TEXT
)
CREATE TABLE "stations" (
"ID" TEXT,
  "LATITUDE" REAL,
  "LONGITUDE" REAL,
  "STNELEV" REAL,
  "NAME" TEXT
)


Part 2
Write a Query Function

In [6]:
def query_climate_database(country, year_begin, year_end, month):
    """
    This function accepts four arguments:

    country, a string giving the name of a country for which data should be returned.
    year_begin and year_end, two integers giving the earliest and latest years for which should be returned.
    month, an integer giving the month of the year for which should be returned.
    
    The return value of query_climate_database() is a Pandas dataframe of temperature readings for the specified country, in the specified date range, in the specified month of the year. 
    """
    
    #SQL command
    cmd = \
    """
    SELECT S.name, T.temp, S.latitude, S.longitude, T.fips, T.year, T.month, C.country
    FROM temperatures T
    LEFT JOIN stations S ON T.id = S.id
    LEFT JOIN countries C ON T.FIPS = C.FIPS
    WHERE C.Country = (?) AND T.year >= (?) AND T.year <= (?) AND T.month = (?)
    """

    Result_df = pd.read_sql_query(cmd, conn, params= (country, year_begin, year_end, month,))
    return (Result_df)




Part 3

In [ ]:
def coef(data_group):
    """
    This function used linear regression to find the coefficient corresponds to the slope of the linear relation.
    """
    x = data_group[["Year"]] 
    y = data_group["Temp"]   
    LR = LinearRegression()
    LR.fit(x, y)
    return LR.coef_[0]




def temperature_coefficient_plot(country, year_begin, year_end, month, min_obs,**kwargs):
    """
    This function accepts five explicit arguments, and an undetermined number of keyword arguments. And the five explicit arguments are: country, year_begin, year_end, and month. This function returns an interactive geographic scatterplot that shows how the average yearly change in temperature varys within a given country.
    """
    
    # I first used query_climate_database function I wrote previously to get a dataset.
    First_DF = query_climate_database(country, year_begin, year_end, month)
    First_DF["Year_count"] = First_DF.groupby(["NAME","Month"])["Year"].transform(len)
    # Applying filter to get rid off stations that do not have enough years of observation data.
    Second_DF = First_DF[First_DF["Year_count"] >= min_obs]
    
    
    coefs = Second_DF .groupby(["NAME"]).apply(coef)
    coefs = coefs.reset_index()
    
    
    finalDF = coefs.merge(Second_DF, on = "NAME")
    finalDF = finalDF.rename(columns = {0:"Estimately Yearly Increase Temperature in Celsius"})
    

    # Graph with scatter_mapbox
    fig = px.scatter_mapbox(finalDF, 
                        lat = "LATITUDE",
                        lon = "LONGITUDE", 
                        hover_name = "NAME", 
                        color = "Estimately Yearly Increase Temperature in Celsius",
                        labels={'Timestep_str':'Date'},
                        title= "Estimates of yearly increase in temperature in " + country + " for stations in month " + str(month) +" from year "+ str(year_begin)+ " to "+ str(year_end), 
                        **kwargs
                        )
    fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
    fig.show()
    write_html(fig, "my_fancy_plot1.html")
 

    
temperature_coefficient_plot("India", 1980, 2020, 1,5, 
                        zoom = 1,
                        opacity = 0.2,
                        height = 300,
                        mapbox_style="carto-positron",)

   

Part 4 Plot 1

This Plot gives the temperature of all stations from a userdefined country during a user defined time frame. This plot shows throughout the years, how the stations' temperatures have changed. I represent each station of each year's temeprature as the average temperature of that station throughout that year (the sum of temperatures from 12 months devides by 12). The plot might seem too overwhelming at first especially when we are looking at a country with many stations. However, one of the nice features of px.line is allowing viewers to click on the names of each line (On the right side of the plot) and make unwanted lines to disappear. This plot is very helpful if one wants to learn the change of temperature of certain stations located in a certain country throughout a certain time frame. I demonstrate the use of my function (temperature_trends_plot) with the data from the stations in India from 1980 to 2020. And only stations with user defined years of observation of temperature data will be included.

In [7]:


def query_climate_database_allyear(country, year_begin, year_end):
    """
    This function accepts three arguments:

    country, a string giving the name of a country for which data should be returned.
    year_begin and year_end, two integers giving the earliest and latest years for which should be returned.

    
    The return value of query_climate_database() is a Pandas dataframe of temperature readings for the specified country, in the specified date range. 
    """
    
    #SQL command
    cmd = \
    """
    SELECT S.name, T.temp, S.latitude, S.longitude, T.fips, T.year, T.month, C.country
    FROM temperatures T
    LEFT JOIN stations S ON T.id = S.id
    LEFT JOIN countries C ON T.FIPS = C.FIPS
    WHERE C.Country = (?) AND T.year >= (?) AND T.year <= (?) 
    """

    Result_df = pd.read_sql_query(cmd, conn, params= (country, year_begin, year_end,))
    return (Result_df)




In [ ]:
def temperature_trends_plot(country, year_begin, year_end, min_obs,**kwargs):
    """
    This function accepts five explicit arguments, and an undetermined number of keyword arguments. And the five explicit arguments are: country, year_begin, year_end, and month. This function returns an interactive geographic scatterplot that shows how the average yearly change in temperature varys within a given country.
    """
    
    # I first used query_climate_database_allyear function I wrote previously to get a dataset.
    First_DF = query_climate_database_allyear(country, year_begin, year_end)
    First_DF["Year_count"] = First_DF.groupby(["NAME","Month"])["Year"].transform(len)
    # Applying filter to get rid off stations that do not have enough years of observation data.
    
    
    Second_DF = First_DF[First_DF["Year_count"] >= min_obs]
    
    time_change = Second_DF.groupby(["NAME","Year"])["Temp"].mean()
    time_change = time_change.reset_index()
    
    fig = px.line(time_change, x="Year", y="Temp", color="NAME", line_group="NAME", hover_name="NAME",title= "Average temperature trends through out a year in " + country + " for all stations from year "+ str(year_begin)+ " to "+ str(year_end))


    fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
    fig.show()
    write_html(fig, "my_fancy_plot2.html")
   

    

   


temperature_trends_plot("India", 1980, 2020,30)




Part 4 Plot 2


This plot helps to visualize the difference in average temperature in a specific user-defined month for all stations in the user-defined countries (need to be input as a list) during a user-defined time frame. In my example, I demonstrate my function by using America from 1950 to 2020. As we can see from the plot, there is a huge difference in average temperature for all stations between the data in 1950 and 2020 in the month of August.

In [ ]:


def query_climate_database_Global(year_begin, year_end, month):
    """
    This function accepts three arguments:

    year_begin and year_end, two integers giving the earliest and latest years for which should be returned.
    month, an integer giving the month of the year for which should be returned.
    
    The return value of query_climate_database() is a Pandas dataframe of temperature readings for the specified country, in the specified date range, in the specified month of the year. 
    """
    
    #SQL command
    cmd = \
    """
    SELECT S.name, T.temp, S.latitude, S.longitude, T.fips, T.year, T.month, C.country
    FROM temperatures T
    LEFT JOIN stations S ON T.id = S.id
    LEFT JOIN countries C ON T.FIPS = C.FIPS
    WHERE T.year >= (?) AND T.year <= (?) AND T.month = (?)
    """

    Result_df = pd.read_sql_query(cmd, conn, params= (year_begin, year_end, month,))
    return (Result_df)



def temperature_difference_plot(ountriesinput,year_begin, year_end, month, min_obs,**kwargs):
    """
    This function accepts five explicit arguments, and an undetermined number of keyword arguments. And the five explicit arguments are: country (user needs to provide in the form of a list), year_begin, year_end, and month. This function returns an interactive geographic scatterplot that shows how the average yearly change in temperature varys within a given country.
    """
    
    # I first used query_climate_database function I wrote previously to get a dataset.
    First_DF = query_climate_database_Global(year_begin, year_end, month)
    First_DF["Year_count"] = First_DF.groupby(["NAME","Month"])["Year"].transform(len)
    # Applying filter to get rid off stations that do not have enough years of observation data.
    Second_DF = First_DF[First_DF["Year_count"] >= min_obs]
    
    # Find the mean temperature of all stations from a user-define country of a user-defined month
    Second_DF = Second_DF.groupby(["Country","Year","Month"])["Temp"].mean()
    Second_DF = pd.DataFrame(Second_DF)
    Second_DF = Second_DF.reset_index()
    # Subset the dataframe according to wanted countries
    Second_DF = Second_DF[Second_DF.Country.isin(ountriesinput)]
    
    # The following steps are to subset dataframe into two dataframes, one with temperature data for all countries at the starting year, one with temperature data for all countries at the ending year. 
    Third_DF = Second_DF.reset_index()
    Begin_DF = Third_DF[Third_DF["Year"] == year_begin]
    Begin_DF = Begin_DF.rename(columns = {"Temp" : "Begin"})
    End_DF = Third_DF[Third_DF["Year"] == year_end]
    End_DF = End_DF.rename(columns = {"Temp" : "End"})
    
    # Merge the two dataframes together and create a new column which is the difference in temperature between the starting year and the ending year.
    FinalDF = End_DF.merge(Begin_DF , on = "Country")
    FinalDF["Difference"] = FinalDF["End"] - FinalDF["Begin"]
  
    # Use a relplot to visualize the difference in temperature
   
    sns.relplot(data = FinalDF, 
            x = "Difference", 
            y = "Country",
            alpha = 0.5, 
            height = 4,
            aspect = 1.7,
            **kwargs
                        )
    
    # Add a straight line x = 0 to see the differences comparing to 0
    plt.plot([0,0], [0,160], color = "Red")
    # Add titles
    labs = plt.gca().set(xlabel = "Difference in Temperature",
                     title = "Difference in Average Temperature in Month " + str(month) + " for All Stations from Year "+ str(year_begin) + " to " + str(year_end))
    
    plt.savefig("plot3.png", bbox_inches = "tight")

    
    
   
    

ountriesinputt = ["United States"]

temperature_difference_plot(ountriesinputt,1950, 2020, 8,5)

   
